In [3]:
import DataSetQuery
import datetime


environmentName = 'DEVv2'
#Sets the URL of the Malard ServiceGateway.
query = DataSetQuery.DataSetQuery('http://localhost:9000',environmentName)

print(query.getEnvironment(environmentName))

#determines the databases that store multiple datasets 
print(query.getParentDataSets())
#retrieve the data sets for a given parent
print(query.getDataSets('mtngla'))
#The overall data cube for the two datasets

parentDs = 'cryotempo'
dataSet = 'GRIS_BASE_CQ2'
region='greenland'

#print(query.getDataSetBoundingBox( parentDs,dataSet,region ))
#print(query.getDataSetBoundingBox( 'mtngla','srtm','himalayas' ))

#Setup the bounding box
minX=0
maxX=100000
minY=100000
maxY=200000
minT=datetime.datetime(2010,12,1,0,0)
maxT=datetime.datetime(2010,12,31,0,0)

print( datetime.datetime.timestamp(minT))
print( datetime.datetime.timestamp(maxT))

#Queries the catalogue and returns the gridcells grouped by x and y with the min and max times.
#gcs = query.getGridCells(  'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT )
#print("Found [%d] Grid cells" % ( len(gcs)) )
#print(gcs)
#Queries the catalogue to determine the shards that comprise the bounding box of interest.
shards = query.getShards( 'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT)
print(shards )





{"name":"DEVv2","maskPublisherPath":"/data/puma1/scratch/v2/malard/mask/","cacheCdfPath":"/data/puma1/scratch/v2/malard/export/","pointCdfPath":"/data/snail1/scratch/v2/malard/points/","mongoConnection":"mongodb://localhost:27018"}
[{"name":"cryotempo"},{"name":"mtngla"}]
[{"name":"publishtest","region":"himalayas"},{"name":"ADwithTDX","region":"alaska"},{"name":"srtm","region":"himalayas"},{"name":"tdx","region":"himalayas"}]
1291161600.0
1293753600.0
1291161600.0
1293753600.0
[{"shardName":"/data/snail1/scratch/v2/mtngla/tdx/swath/y2010/m12/cell_HMA_100000_0/GridCell_0.nc","minX":100000.04722366307,"maxX":196902.89463615167,"minY":5.379702464323978,"maxY":99990.36552472426,"minT":1292154378,"maxT":1292543264,"numberOfPoints":137413},{"shardName":"/data/snail1/scratch/v2/mtngla/tdx/swath/y2010/m12/cell_HMA_-100000_0/GridCell_0.nc","minX":-99867.34617715601,"maxX":-0.23715283634633194,"minY":0.5308469311114532,"maxY":99746.65876281458,"minT":1292672334,"maxT":1293233898,"numberOfPoints

In [ ]:
# loads the json output of the getGridCells call and creates a flattened DataFrame.
from pandas.io.json import json_normalize
import json

data = json.loads(gcs)
df = json_normalize(data)

print(df)

#totalPoints = df['totalPoints'].sum()
#maxNumPoints = df['totalPoints'].max()

#print('TP=[%d] MaxPoints in Cell=[%d]'%(totalPoints,maxNumPoints))   



In [ ]:
#Converts the Json output of the getShards call into a DataFrame.
dataShard = json.loads(shards)
dfShards = json_normalize(dataShard)

print(dfShards['shardName'])


In [ ]:
#Gets the available column names from the NetCDFs that are in the BoundingBox.
print(query.getDataSetColumns( 'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT))

In [ ]:
import MalardHelpers

#Returns a file handle to a NetCDF that is a merge and filter of all the shards in a BoundingBox.
fileName = query.getNetCdfFile( 'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT)

print("File created: %s" % (fileName))

#Converts the NetCDF into a DataFrame.
df = MalardHelpers.getDataFrameFromNetCDF(fileName)

print("Max Elevation %f" % (df['elev'].max()))
print("Min Elevation %f" % (df['elev'].min()))

print("Min Coh %f" % (df['coh'].min()))
print("Min Power %f" % (df['power'].min()))

print(df[0:1])

In [ ]:

#a list of columns to see in the output (x,y,time will be added)
projections = ['elev','power','coh'] 
#a list of dictionaries that specify the filters
#only numeric fields are supported with the following operations gt(greater than), gte ( greater than equals), lt (less than) and lte (less than equals)
#filters are treated as and conditions
filters = [{'column':'power','op':'gt','threshold':10000},{'column':'coh','op':'gt','threshold':0.8}]

#No filters
nofilters = query.executeQuery(  'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT,projections,[])
#Converts the NetCDF into a DataFrame.
dfNoFilter = MalardHelpers.getDataFrameFromNetCDF(nofilters)

print( "Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( dfNoFilter['coh'].max(),  dfNoFilter['coh'].min(), dfNoFilter['power'].max(), dfNoFilter['power'].min(), dfNoFilter['power'].count() ) )  

#With filters
withfilters = query.executeQuery( 'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT,projections,filters)
#Converts the NetCDF into a DataFrame.
withFilter = MalardHelpers.getDataFrameFromNetCDF(withfilters)

print( "Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilter['coh'].max(),  withFilter['coh'].min(), withFilter['power'].max(), withFilter['power'].min(), withFilter['power'].count() ) )  

print( "Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilter['coh'].max(),  withFilter['coh'].min(), withFilter['power'].max(), withFilter['power'].min(), withFilter['power'].count() ) )  


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
## plot with no filters
plt.figure(figsize=(12,10))
plt.scatter(x=dfNoFilter['x'], y=dfNoFilter['y'], c=dfNoFilter['elev'], marker='.', s=20, vmax=7500)
plt.colorbar()
plt.show()

In [ ]:
## plot with filters
plt.figure(figsize=(12,10))
#vmin and vmax set min and max of colour bar
plt.scatter(x=withFilter['x'], y=withFilter['y'], c=withFilter['elev'], marker='.', s=20, vmax=dfNoFilter['elev'].max(), vmin=dfNoFilter['elev'].min())
plt.colorbar()
plt.show()

In [ ]:

#swathDetails - information about the source swath data and how the DataSetLoader did the gridding.
results = query.getSwathDetails('mtngla','tdx','himalayas')

print(results)


In [ ]:
swathById = query.getSwathDetailsFromId('mtngla','tdx','himalayas',1)

print(swathById)
 
swathByName = query.getSwathDetailsFromName('mtngla','tdx','himalayas','CS_LTA__SIR_SIN_2S_20100716T075344_20100716T075554_C001.nc')

print(swathByName)


In [ ]:
#swathDetails - get all the swaths for a particular data set.
import pandas as pd
import MalardHelpers as m

df = m.getSwathDetailsAsDataFrame('mtngla','tdx','himalayas',query)

print(df[0:1])


In [ ]:
sourcePath = '/data/snail1/scratch/test/'#put your input path in here
fileName = 'testmask.shp'#put your test file in here
parentDsName = 'test'
dataSet = 'RS6'
maskType = 'Glacier' 
region = 'HMA' 
minX = 500000 
minY = 0 
size = 100000 

query.publishMask(sourcePath, fileName, parentDsName, dataSet, maskType, region, minX, minY, size )

In [6]:
print(query.getMasks('mtngla'))

[{"dataSet":"SDCv10","type":"Debris","region":"HMA"},{"dataSet":"RGIv60","type":"Glacier","region":"HMA"}]


In [8]:
parentdataset = 'mtngla'
dataSet = 'RGIv60'
maskType = 'Glacier'
region = 'HMA'

print(query.getGridCellMasks(parentdataset, dataSet, maskType, region))

[{"gridCell":{"minX":-1600000,"minY":-100000,"size":100000},"fileName":"/data/puma1/scratch/malard/mask/mtngla/static/RGIv60/Glacier/HMA/cell_x-1600000_y-100000_s100000/mask_Glacier_x-1600000_y-100000_s100000.gpkg"},{"gridCell":{"minX":-1500000,"minY":0,"size":100000},"fileName":"/data/puma1/scratch/malard/mask/mtngla/static/RGIv60/Glacier/HMA/cell_x-1500000_y0_s100000/mask_Glacier_x-1500000_y0_s100000.gpkg"},{"gridCell":{"minX":-1500000,"minY":400000,"size":100000},"fileName":"/data/puma1/scratch/malard/mask/mtngla/static/RGIv60/Glacier/HMA/cell_x-1500000_y400000_s100000/mask_Glacier_x-1500000_y400000_s100000.gpkg"},{"gridCell":{"minX":-1400000,"minY":0,"size":100000},"fileName":"/data/puma1/scratch/malard/mask/mtngla/static/RGIv60/Glacier/HMA/cell_x-1400000_y0_s100000/mask_Glacier_x-1400000_y0_s100000.gpkg"},{"gridCell":{"minX":-1400000,"minY":100000,"size":100000},"fileName":"/data/puma1/scratch/malard/mask/mtngla/static/RGIv60/Glacier/HMA/cell_x-1400000_y100000_s100000/mask_Glacier

In [ ]:
#gets the mask for a gridcell.

print(query.getGridCellMask(parentdataset, dataSet, maskType, region,minX,minY,size))

In [ ]:
parentDsName = 'test'
runName = 'gctest_2'
minX = 0.0
minY = 200000.0
size = 100000.0

#expects a dictionary of string to double.
statistics = {'NumberOfPoints':1000,'MaskedPoints':500}

query.publishGridCellStats( parentDsName, runName, minX, minY, size, statistics )

In [ ]:
#get the available run statistics
parentDsName = 'mtngla'
query.getAvailableRunStatistics( parentDsName )

In [ ]:
#get the statistics for a gridcell.
query.getGridCellStatistics(parentDsName, runName, minX, minY, size)

In [ ]:
query.getRunStatistics(parentDsName,"TestRun")

In [ ]:
#Get the proj4 for code for the given shortCode.
print( query.getProjection('HMA'))

In [ ]:
#Publish points for a grid cell.
parentDataSet = 'mtngla' #intended usage mtngla
dataSet = 'publishtest'
region = 'himalayas'
projection = 'HMA'
sourceFileName = '/data/snail1/scratch/tandemx/swath/y2017/m6/cell_HMA_0_600000/GridCell_0.nc'
minX = 0
minY = 600000
size = 100000

query.publishGridCellPoints(parentDataSet, dataSet,region, minX, minY, size, sourceFileName, projection)


In [ ]:
print(datetime.datetime.fromtimestamp(1337598197))
print(datetime.datetime.fromtimestamp(1279356087))